In [1]:
import torch
import syft as sy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from torch import optim
from tqdm import tqdm
device = torch.device("cuda")
torch.manual_seed(5)

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.empty_cache()

In [5]:
dataset =  pd.read_csv('kdd99.csv', header=None)
dataset.columns = ["c1","c2","c3","c4","c5","c6","c7","c8","c9","c10","c11","c12","c13","c14","c15","c16","c17","c18","c19","c20","c21","c22","c23","c24","c25","c26","c27","c28","c29","j"]
dataset.head()

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c21,c22,c23,c24,c25,c26,c27,c28,c29,j
0,0.0,215.0,45076.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,n
1,0.0,162.0,4528.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,n
2,0.0,236.0,1228.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,n
3,0.0,233.0,2032.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,n
4,0.0,239.0,486.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,n


In [6]:
dataset.replace({"o":0, "n":1}, inplace=True)

In [7]:
dataset = (dataset - dataset.min())/(dataset.max() - dataset.min())

In [8]:
dataset.head()

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c21,c22,c23,c24,c25,c26,c27,c28,c29,j
0,0.0,0.003942,0.003843,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.002970,0.000386,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.003922,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.004327,0.000105,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.007843,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.004272,0.000173,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.011765,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.004382,0.000041,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.015686,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
set(dataset.j)

{0.0, 1.0}

In [10]:
X, y = Variable(torch.from_numpy(dataset[[cols for cols in dataset.columns if cols!='j']].values)).cuda(), Variable(torch.from_numpy(dataset.j.values)).cuda()

In [11]:
data_client1 = dataset.sample(frac=0.5)
data_client2 = dataset[~dataset.index.isin(data_client1.index)]

In [12]:
features_client1, target_client1 = data_client1[[cols for cols in data_client1.columns if cols!='j']], data_client1.j
features_client2, target_client2 = data_client2[[cols for cols in data_client2.columns if cols!='j']], data_client2.j

In [13]:
x_client1 = Variable(torch.from_numpy(features_client1.values), requires_grad = False).cuda().float()
x_client2 = Variable(torch.from_numpy(features_client2.values), requires_grad = False).cuda().float()

y_client1 = Variable(torch.from_numpy(target_client1.values), requires_grad = False).cuda().float()
y_client2 = Variable(torch.from_numpy(target_client2.values), requires_grad = False).cuda().float()

In [14]:
def model(In_size, H_size=14):
    if type(H_size) is not list:
        model = torch.nn.Sequential(
                    torch.nn.Linear(In_size, H_size),
                    torch.nn.ReLU(),
                    torch.nn.Linear(H_size, H_size),
                    torch.nn.Tanh(),
                    torch.nn.Linear(H_size, H_size),
                    torch.nn.ReLU(),
                    torch.nn.Linear(H_size, In_size))
            
    
    return model
    
loss_fn = torch.nn.MSELoss(reduction='sum')

In [15]:
auto_mod = model(dataset.shape[1] - 1, 14).cuda()

opt_auto = optim.SGD(params=auto_mod.parameters(), lr=1e-5)
loss_auto = torch.nn.MSELoss(reduction = 'sum')

In [16]:
print(auto_mod)

Sequential(
  (0): Linear(in_features=29, out_features=14, bias=True)
  (1): ReLU()
  (2): Linear(in_features=14, out_features=14, bias=True)
  (3): Tanh()
  (4): Linear(in_features=14, out_features=14, bias=True)
  (5): ReLU()
  (6): Linear(in_features=14, out_features=29, bias=True)
)


In [17]:
def classification_model(In_size, H_size=10, out_size=1):
    model = torch.nn.Sequential(
                    torch.nn.Linear(In_size, H_size),
                    torch.nn.ReLU(),
                    torch.nn.Linear(H_size, H_size),
                    torch.nn.Tanh(),
                    torch.nn.Linear(H_size, out_size),
                    torch.nn.Sigmoid())
    return model


In [18]:
class_mod = classification_model(dataset.shape[1] - 1).cuda()
opt_class = optim.SGD(params=class_mod.parameters(), lr=1e-5)
loss_class = torch.nn.BCELoss().cuda()

In [19]:
print(class_mod)

Sequential(
  (0): Linear(in_features=29, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Tanh()
  (4): Linear(in_features=10, out_features=1, bias=True)
  (5): Sigmoid()
)


In [20]:
hook = sy.TorchHook(torch)

client_1 = sy.VirtualWorker(hook, id="client_1")
client_2 = sy.VirtualWorker(hook, id="client_2")

In [21]:
epoch = 20
batch_size= 256
batches = []
for i in range(0, len(x_client2), batch_size):
    batches.append((x_client1[i:( i + batch_size)].send(client_1), y_client1[i:( i + batch_size)].send(client_1)))
    batches.append((x_client2[i:( i + batch_size)].send(client_2), y_client2[i:( i + batch_size)].send(client_2)))

RuntimeError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 4.00 GiB total capacity; 3.06 GiB already allocated; 14.80 MiB free; 16.07 MiB cached)

In [ ]:
for __ in range(epoch):
    print('Epoch start\n')
    losses= []
    
    for batch in tqdm(batches):
        
        worker = batch[0].location
        auto_mod.send(worker)
        opt_auto.zero_grad()
        pred = auto_mod(batch[0])
        loss_val = loss_auto(pred, batch[0])
        loss_val.backward()
        
        opt_auto.step()
        auto_mod.get()
        losses.append(loss_val.get())
    
    print(sum(losses)/len(losses))



In [ ]:
x_client1 = auto_mod(Variable(torch.from_numpy(features_client1.values), requires_grad = False).cuda().float())
x_client2 = auto_mod(Variable(torch.from_numpy(features_client2.values), requires_grad = False).cuda().float())

In [ ]:
epoch = 20
batch_size= 256
batches = []
for i in range(0, len(x_client2), batch_size):
    batches.append((x_client1[i:( i + batch_size)].send(client_1), y_client1[i:( i + batch_size)].send(client_1)))
    batches.append((x_client2[i:( i + batch_size)].send(client_2), y_client2[i:( i + batch_size)].send(client_2)))

In [ ]:
for __ in range(epoch):
    print('Epoch start\n')
    losses= []
    for batch in tqdm(batches):
        worker = batch[0].location
        class_mod.send(worker)
        opt_class.zero_grad()
        pred = class_mod(batch[0])
        loss_val = loss_class(pred, batch[1])
        loss_val.backward()
        opt_class.step()
        class_mod.get()
        losses.append(loss_val.get())
    
    print(sum(losses)/len(losses))



In [ ]:
X = Variable(torch.from_numpy(dataset[[cols for cols in dataset.columns if cols!='j']].values.astype('float'))).cuda()

In [ ]:
pred1 = np.array(class_mod(x_client1).cpu().detach())
pred2 = np.array(class_mod(x_client2).cpu().detach())

In [ ]:
pred = np.vstack([pred1.reshape(-1, 1), pred2.reshape(-1, 1)])
pred = pred.reshape(-1,1)
pred = (pred - pred.min())/(pred.max() - pred.min())
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.reshape(-1, 1)
target = np.vstack([y_client1.cpu().numpy().reshape(-1, 1), y_client2.cpu().numpy().reshape(-1, 1)])

In [ ]:
print(classification_report(pred, target))

In [ ]:
mat = confusion_matrix(pred, target)

In [ ]:
plt.matshow(mat)
plt.colorbar()

In [ ]:
losses = [x.item() for x in losses]

In [ ]:
plt.plot(losses)
plt.title("Loss vs iteration")
plt.xlabel('iterations')
plt.ylabel('loss')